# Logistic Regression: Predicting Survey Respondent's Happiness

### - Bassam Atheeque
---

## Data description: [Data Source](http://archive.ics.uci.edu/ml/datasets/Somerville+Happiness+Survey) from UCI Machine Learning Repository

The City of Somerville sends out a happiness survey to a random sample of Somerville residents asking them to rate their personal happiness.

**happy**: Whether the survey respondent is happy or not

**city_services**: The availability of information about the city services

**housing_cost**: The cost of housing

**public_schools**: The overall quality of public schools

**trust_police**: Your trust in the local police

**streets_sidewalk**: The maintenance of streets and sidewalks

**social_events**: The availability of social community events

In [1]:
# Importing the libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
emotion = pd.read_csv('./happiness.csv')

In [3]:
emotion.head()

,happy,city_services,housing_cost,public_schools,trust_police,streets_sidewalk,social_events
0,No,3,2,3,4,2,4
1,No,3,2,3,3,4,3
2,Yes,5,3,5,3,5,5
3,No,2,4,3,3,3,5
4,No,5,4,3,1,3,5


In [4]:
emotion['happy'].value_counts()

Yes    77
No     66
Name: happy, dtype: int64

In [5]:
perc_happy = len(emotion[emotion['happy']=='Yes'])/len(emotion)

print (round((perc_happy*100),2), '% of the respondents in the data set were happy.')

53.85 % of the respondents in the data set were happy.


### Checking for missing scores in the data:

In [6]:
emotion.dtypes

happy               object
city_services        int64
housing_cost        object
public_schools      object
trust_police        object
streets_sidewalk     int64
social_events        int64
dtype: object

#### Converting Object to Integers:

In [7]:
emotion[['housing_cost', 'public_schools', 'trust_police']]=  emotion[['housing_cost',
                                                                       'public_schools',
                                                                       'trust_police']].apply(pd.to_numeric, 
                                                                                              errors='coerce', 
                                                                                              downcast='integer')

In [8]:
# Reconfirming the data types.

emotion.dtypes

happy                object
city_services         int64
housing_cost        float64
public_schools      float64
trust_police        float64
streets_sidewalk      int64
social_events         int64
dtype: object

In [9]:
# Checking missing values

emotion.isna().sum()

happy               0
city_services       0
housing_cost        1
public_schools      1
trust_police        2
streets_sidewalk    0
social_events       0
dtype: int64

In [10]:
emotion.shape

(143, 7)

In [11]:
#deleting empty rows

emotion = emotion.dropna()

In [12]:
emotion.shape

(140, 7)

### Using a pivot table to check the average scores (in each of the six categories) for the respondents who were happy and not happy.

In [13]:
emotion.head(2)

,happy,city_services,housing_cost,public_schools,trust_police,streets_sidewalk,social_events
0,No,3,2.0,3.0,4.0,2,4
1,No,3,2.0,3.0,3.0,4,3


In [14]:
# Pivot Table:

pd.pivot_table (emotion, values = ['city_services', 'housing_cost', 'public_schools', 'trust_police', 
                                   'streets_sidewalk','social_events'], 
                index = 'happy',
                aggfunc = [np.mean]).round(2)

mean                                            \
      city_services housing_cost public_schools social_events   
happy                                                           
No             3.77         2.46           2.94          4.02   
Yes            4.56         3.19           3.65          4.39   

                                     
      streets_sidewalk trust_police  
happy                                
No                3.05         3.45  
Yes               4.01         3.77

### Using Statsmodels to fit a logistic regression model to the data with all six scores as predictors.

In [15]:
import statsmodels.formula.api as smf
from scipy import stats

In [16]:
#Converting categorical column to numerical

binary_happy = pd.get_dummies(emotion['happy'], drop_first = True)
binary_happy.head()

# 'No' is 0
#' Yes' is 1

,Yes
0,0
1,0
2,1
3,0
4,0


In [17]:
# Adding the binary column to the data:

emotion['binary_happy']=binary_happy 

In [18]:
emotion.head()

,happy,city_services,housing_cost,public_schools,trust_police,streets_sidewalk,social_events,binary_happy
0,No,3,2.0,3.0,4.0,2,4,0
1,No,3,2.0,3.0,3.0,4,3,0
2,Yes,5,3.0,5.0,3.0,5,5,1
3,No,2,4.0,3.0,3.0,3,5,0
4,No,5,4.0,3.0,1.0,3,5,0


In [19]:
predictors = 'binary_happy ~ city_services + housing_cost+ public_schools+ trust_police+ streets_sidewalk +social_events'

emotion_stats = smf.logit (formula = predictors , data = emotion).fit()
print (emotion_stats.summary())

Optimization terminated successfully.
         Current function value: 0.458132
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           binary_happy   No. Observations:                  140
Model:                          Logit   Df Residuals:                      133
Method:                           MLE   Df Model:                            6
Date:                Wed, 02 Feb 2022   Pseudo R-squ.:                  0.3366
Time:                        20:17:11   Log-Likelihood:                -64.138
converged:                       True   LL-Null:                       -96.683
Covariance Type:            nonrobust   LLR p-value:                 4.136e-12
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -11.7355      2.376     -4.939      0.000     -16.393      -7.078
city_servic

### Explanation: 
#### The scores of **trust_police , social_events** are not significant at a 95% confidence level as their confidence interval has a negative lower bound value to positive upper bound value and thus, zero falls between them. Also, their p-value is greater than our significance level, 𝛼 = 0.05
#### Whereas the scores of 'city_services', 'housing_cost', 'public_schools' and 'streets_sidewalk' are the significant predictors as their p-value is lesser than 𝛼 = 0.05

### Dropping all the predictors that were not significant and fitting a new model with the remaining predictors.

In [20]:
signi_predictors = 'binary_happy ~ city_services + housing_cost+ public_schools+ + streets_sidewalk'

signi_emotion_stats = smf.logit (formula = signi_predictors , data = emotion).fit()
print (signi_emotion_stats.summary())

Optimization terminated successfully.
         Current function value: 0.463724
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           binary_happy   No. Observations:                  140
Model:                          Logit   Df Residuals:                      135
Method:                           MLE   Df Model:                            4
Date:                Wed, 02 Feb 2022   Pseudo R-squ.:                  0.3285
Time:                        20:17:11   Log-Likelihood:                -64.921
converged:                       True   LL-Null:                       -96.683
Covariance Type:            nonrobust   LLR p-value:                 5.265e-13
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -10.5915      1.879     -5.638      0.000     -14.273      -6.910
city_servic

In [21]:
params = signi_emotion_stats.params
params

Intercept          -10.591540
city_services        1.149402
housing_cost         0.652744
public_schools       0.624763
streets_sidewalk     0.560722
dtype: float64

## Fitted Model:

###    $$\frac{p(x)}{1-p(x)}=e^{\beta_0+\beta_1x}$$

###    $$\frac{p(x)}{1-p(x)}=e^{-10.6167 + \text{(1.1515 * city_services)} + \text{(0.6544 * housing_cost)} + \text{(0.6272 * public_schools)} + \text{(0.5620 * streets_sidewalk)}} $$

In [22]:
# Finding the exponents of the parameters :

print ('beta_0:', np.exp(params[0]).round(8))
print ('beta_1:', np.exp(params[1]).round(3))
print ('beta_2:', np.exp(params[2]).round(3))
print ('beta_3:', np.exp(params[3]).round(3))
print ('beta_4:', np.exp(params[4]).round(3))

beta_0: 2.513e-05
beta_1: 3.156
beta_2: 1.921
beta_3: 1.868
beta_4: 1.752


### Interpretation:

#### For the city_services predictor, its associated estimated coefficient is 3.156.
#### This means that based on our logistic regression model, if we increase the city_services rating by 1, the probability of being happy will be increased by a ratio of 3.156

## Train-Test Split method
#### We will be using the train/test split method from scikit-learn by splitting the data to 75% (training set) and 25% (test set) and the random_state set to 11.

In [23]:
emotion.head(2)

,happy,city_services,housing_cost,public_schools,trust_police,streets_sidewalk,social_events,binary_happy
0,No,3,2.0,3.0,4.0,2,4,0
1,No,3,2.0,3.0,3.0,4,3,0


#### Using train_test_split to split the data:

In [24]:
y = emotion['binary_happy']

X = emotion[['city_services', 'housing_cost', 'public_schools', 'trust_police', 'streets_sidewalk','social_events']]

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 11)

### Fitting the logistic regression model:

In [26]:
from sklearn.linear_model import LogisticRegression

lor = LogisticRegression()

lor.fit(X_train, y_train)

LogisticRegression()

### Making predictions from our model:

In [27]:
# Predicition Probability with the test data:

y_predprob = lor.predict_proba(X_test)[:,1].round(3)
y_predprob 

array([0.677, 0.15 , 0.883, 0.851, 0.869, 0.859, 0.097, 0.771, 0.035,
       0.4  , 0.869, 0.697, 0.374, 0.936, 0.63 , 0.777, 0.249, 0.728,
       0.977, 0.911, 0.888, 0.078, 0.709, 0.928, 0.368, 0.821, 0.75 ,
       0.944, 0.848, 0.911, 0.872, 0.953, 0.64 , 0.006, 0.466])

In [28]:
# Predicition with the test data:

y_pred = lor.predict (X_test)
y_pred

array([1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=uint8)

In [29]:
# Actual data, Predicted data, Associated Probability of being happy

np.column_stack((y_test, y_pred, y_predprob))

array([[1.   , 1.   , 0.677],
       [0.   , 0.   , 0.15 ],
       [1.   , 1.   , 0.883],
       [1.   , 1.   , 0.851],
       [1.   , 1.   , 0.869],
       [0.   , 1.   , 0.859],
       [0.   , 0.   , 0.097],
       [0.   , 1.   , 0.771],
       [0.   , 0.   , 0.035],
       [1.   , 0.   , 0.4  ],
       [1.   , 1.   , 0.869],
       [1.   , 1.   , 0.697],
       [0.   , 0.   , 0.374],
       [1.   , 1.   , 0.936],
       [1.   , 1.   , 0.63 ],
       [0.   , 1.   , 0.777],
       [0.   , 0.   , 0.249],
       [0.   , 1.   , 0.728],
       [1.   , 1.   , 0.977],
       [1.   , 1.   , 0.911],
       [1.   , 1.   , 0.888],
       [0.   , 0.   , 0.078],
       [1.   , 1.   , 0.709],
       [1.   , 1.   , 0.928],
       [0.   , 0.   , 0.368],
       [0.   , 1.   , 0.821],
       [0.   , 1.   , 0.75 ],
       [1.   , 1.   , 0.944],
       [1.   , 1.   , 0.848],
       [1.   , 1.   , 0.911],
       [0.   , 1.   , 0.872],
       [1.   , 1.   , 0.953],
       [1.   , 1.   , 0.64 ],
       [0.

## Confusion Matrix:

In [30]:
from sklearn import metrics

conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat

array([[ 9,  7],
       [ 1, 18]], dtype=int64)

In [31]:
# Confusion Matrix in dataframe format:

dfconfmat = pd.DataFrame(data = conf_mat, columns = ['pred: No', 'pred: Yes'], index = ['test: No','test: Yes'])
dfconfmat

,pred: No,pred: Yes
test: No,9,7
test: Yes,1,18


In [32]:
# True Negative
TN = dfconfmat['pred: No'][0]

# True Positive
TP = dfconfmat['pred: Yes'][1]

# False Negative
FN = dfconfmat['pred: No'][1]

# False Positive
FP = dfconfmat['pred: Yes'][0]

## Metrics:

In [33]:
# Accuracy Check:

accuracy = metrics.accuracy_score (y_test, y_pred)

print ('The accuracy is:' , accuracy)

The accuracy is: 0.7714285714285715


In [34]:
# Precision:

precision = TP / (FP + TP)

print ('The precision is:' , precision)

The precision is: 0.72


In [35]:
# Recall

recall = TP / (TP + FN)
print ('The recall is:' , recall)

The recall is: 0.9473684210526315


In [36]:
# Specificity:

specificity = TN / (TN + FP)
print ('The specificity is:' , specificity)

The specificity is: 0.5625


In [37]:
# False ALarm Rate:

F_A_R = FP / (FP + TN)
print ('False ALarm Rate:' , F_A_R )

False ALarm Rate: 0.4375


In [38]:
# F1 score:
# It is interpreted as a harmonic mean of the precision and recall

F1 = (2 * ((precision * recall) / (precision + recall))).round(3)
print ('F1 score:' , F1 )

F1 score: 0.818


### Using the model to predict based on the input data given below:


|| city_services | housing_cost | public_schools | trust_police | streets_sidewalk | social_events |
|-|-|-|-|-|-|-|
|Eleanor|4|2|3|5|4|4|
| Chidi |4|4|4|2|4|1|

#### Predicting by creating a dataframe of the input data:

In [39]:
test_dict = {'city_services':[4,4], 'housing_cost':[2,4],'public_schools':[3,4],'trust_police':[5,2],'streets_sidewalk':[4,4],'social_events':[4,1]}
input_test = pd.DataFrame(test_dict)

input_test

,city_services,housing_cost,public_schools,trust_police,streets_sidewalk,social_events
0,4,2,3,5,4,4
1,4,4,4,2,4,1


In [40]:
print (lor.predict_proba(input_test))
print (lor.predict(input_test))

[[0.44382641 0.55617359]
 [0.27922531 0.72077469]]
[1 1]


### Based on the scores they gave, our model's output: '1' which is 'Yes' predicted that both Elenaor and Chidi are happy. The probability percentage of our model for them being happy was 55.6% and 79.6% respectively.

## Cross-Validation
#### Using 5-fold cross-validation to compare the model with all six predictors with the model after dropping the insignificant predictors.

In [41]:
emotion.head()

,happy,city_services,housing_cost,public_schools,trust_police,streets_sidewalk,social_events,binary_happy
0,No,3,2.0,3.0,4.0,2,4,0
1,No,3,2.0,3.0,3.0,4,3,0
2,Yes,5,3.0,5.0,3.0,5,5,1
3,No,2,4.0,3.0,3.0,3,5,0
4,No,5,4.0,3.0,1.0,3,5,0


In [42]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

crossLOR = LogisticRegression()

#### Model 1 (with all predictors):

In [43]:
X_allpred = emotion[['city_services', 'housing_cost', 'public_schools', 'trust_police', 'streets_sidewalk','social_events']]

In [44]:
y_allpred = emotion['binary_happy']

In [45]:
Model1 = cross_val_score ( crossLOR, X_allpred, y_allpred, cv = 5, scoring = 'accuracy').mean()

print ((Model1*100).round(2), ('% is the accuracy for our Model 1.'))

71.43 % is the accuracy for our Model 1.


#### Model 2 (after dropping insignificant predictors):

In [46]:
X_sigpred = emotion[['city_services', 'housing_cost', 'public_schools', 'streets_sidewalk']]

In [47]:
y_sigpred = emotion['binary_happy']

In [48]:
Model2 = cross_val_score ( crossLOR, X_sigpred, y_sigpred, cv = 5, scoring = 'accuracy').mean()

print ((Model2*100).round(2), ('% is the accuracy for our Model 2.'))

75.71 % is the accuracy for our Model 2.


### Based on the model accuracy, Model 2 is better as it has a accuracy of 75.71% when compared to Model 1 which has a accuracy of 71.43%
### It is because **Model 2** doesn't consist of non-signficant predictors. Significant predictors are those whose p-value was lesser than 𝛼 = 0.05

## Thank You!
---